# Task: Local LLM Integration & Recipe Chatbot

## Objective
Set up an open-source AI model locally, prepare a custom recipe dataset, expose the model via an API,
and build a chatbot that suggests recipes based on user-provided ingredients.

Example:
- Input: Egg, Onion
- Output: A recipe suggestion using egg and onion


## Approach Used

1. Use a lightweight open-source LLM suitable for low-resource environments.
2. Prepare a custom recipe dataset for domain-specific behavior.
3. Use prompt-based tuning instead of heavy fine-tuning.
4. Expose the model via a FastAPI REST API.
5. Build a simple chatbot interface (CLI-style).
6. Ensure all outputs are returned in JSON format.

This solution is lightweight, reproducible, and works on standard systems.


In [1]:
!pip install fastapi uvicorn transformers torch requests


## Custom Recipe Dataset

This dataset is used to guide the model’s behavior and represents domain-specific training data.


In [2]:
import json

recipes = [
    {
        "ingredients": "egg, onion",
        "recipe": "Egg Onion Omelette: Beat eggs, sauté chopped onions, pour eggs, cook until fluffy."
    },
    {
        "ingredients": "potato, onion",
        "recipe": "Aloo Pyaz Sabzi: Fry onions, add potatoes, spices, and cook until soft."
    },
    {
        "ingredients": "tomato, egg",
        "recipe": "Tomato Egg Bhurji: Cook tomatoes with spices, add eggs, scramble well."
    },
    {
        "ingredients": "rice, vegetables",
        "recipe": "Vegetable Fried Rice: Stir-fry vegetables, add cooked rice, soy sauce, and mix well."
    }
]

with open("recipes.json", "w") as f:
    json.dump(recipes, f, indent=2)

print("Custom recipe dataset created.")


Custom recipe dataset created.


In [3]:
import json

with open("recipes.json", "r") as f:
    RECIPE_DB = json.load(f)

print("Recipe dataset loaded. Total recipes:", len(RECIPE_DB))


Recipe dataset loaded. Total recipes: 4


## Model Selection

We use **google/flan-t5-small** because:
- Open-source
- Lightweight
- Works on CPU
- Designed for instruction-based tasks
- Suitable for constrained hardware


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

print("Model loaded successfully.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded successfully.


## Recipe Generation Logic

The model is prompted using ingredients provided by the user.


In [5]:
import re

def normalize_ingredients(ingredients: str):
    """
    Normalize ingredient input:
    - lowercase
    - comma / space independent
    - handle plural forms (eggs -> egg)
    """
    ingredients = ingredients.lower()
    ingredients = ingredients.replace(",", " ")
    tokens = re.split(r"\s+", ingredients)

    normalized = set()
    for token in tokens:
        token = token.strip()
        if not token:
            continue
        if token.endswith("s"):
            token = token[:-1]
        normalized.add(token)

    return normalized


In [6]:
def generate_recipe(ingredients: str) -> str:
    """
    Generate a recipe based on user-provided ingredients.

    Strategy:
    1. Normalize input ingredients (order & plural safe)
    2. Match against custom dataset using maximum overlap
    3. Require at least 60% ingredient overlap
    4. Fall back to controlled LLM generation
    5. Guard against prompt-echo / low-quality outputs
    """

    # Normalize user input
    user_ingredients = normalize_ingredients(ingredients)

    # Safety check
    if not user_ingredients:
        return "Please provide at least one valid ingredient."

    # 1️⃣ Dataset-based retrieval (max overlap)
    best_match = None
    best_overlap = 0

    for item in RECIPE_DB:
        recipe_ingredients = normalize_ingredients(item["ingredients"])
        overlap = len(user_ingredients & recipe_ingredients)

        if overlap > best_overlap:
            best_overlap = overlap
            best_match = item

    # Require at least 60% ingredient match
    if best_match and (best_overlap / len(user_ingredients)) >= 0.6:
        return best_match["recipe"]

    # 2️⃣ Controlled LLM fallback
    prompt = (
        "You are a cooking assistant.\n"
        f"Ingredients available: {', '.join(sorted(user_ingredients))}\n"
        "Rules:\n"
        "- Use ONLY the given ingredients\n"
        "- Do NOT mention ingredients that are not listed\n"
        "- If a proper recipe is not possible, say so politely\n"
        "- Keep the answer short and practical\n\n"
        "Recipe:"
    )

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=120,
        num_beams=4,
        early_stopping=True
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 3️⃣ Guardrails against prompt echo / bad outputs
    bad_phrases = [
        "keep the answer short",
        "use only the given ingredients",
        "do not mention ingredients",
        "rules:",
        "recipe:"
    ]

    if not response or any(bp in response.lower() for bp in bad_phrases):
        return "A proper recipe cannot be made using only the given ingredients."

    return response


## API Integration Using FastAPI

The API accepts ingredients and returns a recipe in JSON format.


In [7]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="Recipe Chatbot API")

class RecipeRequest(BaseModel):
    ingredients: str

@app.post("/get-recipe")
def get_recipe(request: RecipeRequest):
    recipe = generate_recipe(request.ingredients)

    source = "dataset" if recipe in [r["recipe"] for r in RECIPE_DB] else "llm"

    return {
        "ingredients": request.ingredients,
        "recipe": recipe,
        "source": source
    }


## Running the API Server

We use `uvicorn` to start the FastAPI server.


In [8]:
import nest_asyncio
import uvicorn
import threading

nest_asyncio.apply()

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Run uvicorn in a separate thread
thread = threading.Thread(target=run_uvicorn)
thread.start()

print("FastAPI server started in a background thread on http://0.0.0.0:8000")
print("You can interact with the API by sending requests to this address.")

FastAPI server started in a background thread on http://0.0.0.0:8000
You can interact with the API by sending requests to this address.


### Note on Server Execution in Colab

The FastAPI server is started in a background thread to avoid blocking the notebook runtime.
This allows further interaction with the API and chatbot within the same notebook.


## Chatbot Interface (CLI Style)

The chatbot sends ingredient input to the API and displays responses conversationally.


In [9]:
import requests

def chatbot_cli():
    while True:
        ingredients = input("Enter ingredients (or type exit): ")
        if ingredients.lower() == "exit":
            break

        response = requests.post(
            "http://127.0.0.1:8000/get-recipe",
            json={"ingredients": ingredients}
        )

        print("\nRecipe Suggestion:")
        print(response.json()["recipe"])
        print("-" * 40)

# Uncomment below line to use interactively
chatbot_cli()


INFO:     Started server process [712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Enter ingredients (or type exit): onion tomato eggs chili
INFO:     127.0.0.1:36900 - "POST /get-recipe HTTP/1.1" 200 OK

Recipe Suggestion:
A proper recipe cannot be made using only the given ingredients.
----------------------------------------
Enter ingredients (or type exit): garlic cheese
INFO:     127.0.0.1:44458 - "POST /get-recipe HTTP/1.1" 200 OK

Recipe Suggestion:
A proper recipe cannot be made using only the given ingredients.
----------------------------------------
Enter ingredients (or type exit): eggs onion
INFO:     127.0.0.1:35956 - "POST /get-recipe HTTP/1.1" 200 OK

Recipe Suggestion:
Egg Onion Omelette: Beat eggs, sauté chopped onions, pour eggs, cook until fluffy.
----------------------------------------
Enter ingredients (or type exit): exit


### Example

**Input:** egg, onion  
**Output:** Egg Onion Omelette with simple cooking steps


## Scalability & Future Improvements

- Can be deployed as a microservice
- Can use larger models on better hardware
- Can be enhanced with vector search for ingredient matching
- Can be integrated with a web or mobile UI
